In [30]:
# Importing libraries
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [31]:
from sklearn import cross_validation

# Load data. Skip column 1 because date format skip the date format (column 1) because it doesn't work

# X = np.loadtxt('Data/train.csv',  delimiter=',', skiprows=1, usecols=range(2, 14))
# y = np.loadtxt('Data/train.csv', delimiter=',', skiprows=1, usecols=[14])
# finaltestX = np.loadtxt('Data/test.csv', delimiter=',', skiprows=1, usecols=range(2,14))

# Removed variable 'atemp' for possibly excessive overlap of features (gives slightly better results)
X = np.loadtxt('Data/train_noatemp.csv',  delimiter=',', skiprows=1, usecols=range(2, 13))
y = np.loadtxt('Data/train_noatemp.csv', delimiter=',', skiprows=1, usecols=[13])
X_kaggle = np.loadtxt('Data/test.csv', delimiter=',', skiprows=1, usecols=range(2,13))

"""Split class lables into intervals of 10 to experiment stratified fold cross-validation?"""
# Still doesn't work; not enough samples for each interval

# j = 0
# k = 0
# for j in range(100):       # Record no. of bicycles was 977
#     for i in range(len(y)):
#         if j*10 <= y[i] <= (j+1)*10:
#             y[i] = j*10 + 10
# folds = cross_validation.StratifiedKFold(y, n_folds=10, shuffle=True)
# print(folds)

'Split class lables into intervals of 10 to experiment stratified fold cross-validation?'

In [34]:
# split data for cross-validation

from sklearn.cross_validation import KFold
from sklearn import linear_model

number_of_folds = 10
kfolds = KFold(n=len(X), n_folds = number_of_folds, shuffle=True)

print("LINEAR REGRESSION - CROSS VALIDATION ON TRAINING DATA SET:\n")

# checking performance on training data (linreg + cross validation)
def cross_validate(design_matrix, labels, classifier, cv_folds):
    pred = np.zeros(labels.shape)             
    coeff = np.zeros((number_of_folds, X.shape[1]))    
    # store trained linreg coefficients
    classifier.fit(design_matrix, labels)            

    foldcount = 1
    for tr, te in cv_folds:
        print('train%d:' % foldcount, tr, 'test%d:' % foldcount, te)
        print('shapes%d:' % foldcount, tr.shape, te.shape)

        classifier.fit(design_matrix[tr], labels[tr])             
        pred[te] = classifier.predict(design_matrix[te])    
        pred[te] = pred[te].astype(int)                            # convert predictions to integers
        #print('pred%d:' % foldcount, pred[te])
        coeff[foldcount-1] = classifier.coef_
        #print('coefficients%d:' % foldcount, classifier.coef_)
        foldcount += 1
    return pred, coeff

lr = linear_model.LinearRegression()          # initiate model object
result = cross_validate(X, y, lr, kfolds)
#print(result[0], result[1])

result1 = result[0]

coefficients = result[1]                      # extract the coefficient matrix
coeff_mean = np.mean(coefficients, axis=0)    # get average for each column
#print(coeff_mean)

# make a second round of predictions using the average coefficients
result2 = (X*coeff_mean).astype(int)
result2 = np.sum(result2, axis=1)
#print(result2)

# compute the number of incorrect results
print("\nNumber of mislabeled points out of a total %d points: %d" % (X.shape[0], (y != result1).sum()))
close = 1
for nn in range(5):           # test how close is the result up to +/- 50
    neighbor = close*10
    print("Number of close points within +/-%d: %d" % (neighbor, ((y >= result1-neighbor) & (y <= result1+neighbor)).sum()))
    close += 1

print("Number of mislabeled points out of a total %d points: %d (using average coefficients)" % \
      (X.shape[0], (y != result2).sum()))

close = 1
for nn in range(5):         
    neighbor = close*10
    print("Number of close points within +/-%d: %d (using average coefficients)" % \
          (neighbor, ((y >= result2-neighbor) & (y <= result2+neighbor)).sum()))
    close += 1

"""Conclusion: Linear regression sucks?"""

print("\nLINEAR REGRESSION ON TEST DATA FOR SUBMISSION:\n")
# for submission to Kaggle
lr_1 = linear_model.LinearRegression()          # initiate model object
lr_1.fit(X, y)
pred_1 = lr_1.predict(X_kaggle)
pred_1 = pred_1.astype(int)
print('coefficients:', lr_1.coef_)
print('pred_1:', pred_1)
# alternatively, use the coefficients to make prediction
# Note: lr_1.coef_ is almost the same as coeff_mean obtained previously)
pred_2 = (X_kaggle*lr_1.coef_).astype(int)
pred_2 = np.sum(pred_2, axis=1)
print('pred_2:', pred_2)

# np.savetxt('Data/linreg_mod1.csv', pred_1, delimiter=',')
# np.savetxt('Data/linreg_mod2.csv', pred_2, delimiter=',')

"""NEXT WORK: TRY REMOVING SOME VARIABLES THAT HAVE VERY SMALL COEFFICIENTS TO IMPROVE LINREG..
OR REMOVE ONES WITH EXCESSIVELY(?) LARGE NEGATIVE COEFFICIENTS"""


LINEAR REGRESSION - CROSS VALIDATION ON TRAINING DATA SET:

train1: [    0     1     2 ..., 10882 10884 10885] test1: [    9    12    14 ..., 10843 10877 10883]
shapes1: (9797,) (1089,)
train2: [    0     1     2 ..., 10882 10883 10884] test2: [   18    19    26 ..., 10856 10869 10885]
shapes2: (9797,) (1089,)
train3: [    0     1     2 ..., 10883 10884 10885] test3: [    5     8    24 ..., 10868 10876 10882]
shapes3: (9797,) (1089,)
train4: [    0     1     2 ..., 10882 10883 10885] test4: [    4    11    15 ..., 10851 10852 10884]
shapes4: (9797,) (1089,)
train5: [    0     1     2 ..., 10883 10884 10885] test5: [   20    29    53 ..., 10835 10844 10848]
shapes5: (9797,) (1089,)
train6: [    1     3     4 ..., 10883 10884 10885] test6: [    0     2     6 ..., 10847 10864 10872]
shapes6: (9797,) (1089,)
train7: [    0     1     2 ..., 10883 10884 10885] test7: [    3    10    21 ..., 10837 10850 10853]
shapes7: (9798,) (1088,)
train8: [    0     1     2 ..., 10883 10884 10885] test8: 

'NEXT WORK: TRY REMOVING SOME VARIABLES THAT HAVE VERY SMALL COEFFICIENTS TO IMPROVE LINREG..\nOR REMOVE ONES WITH EXCESSIVELY(?) LARGE NEGATIVE COEFFICIENTS'